# Parallel Chains

Similarly to how sequential chains connect chains in series, parallel chains connect chains in parallel. This is a useful abstraction when you want to independently process the same input with multiple different chains, possibly asynchronously.

In the following examples, we wlll show:
- how we can use `ParallelChain` to take a list of chains and apply each independently to the same input.
- how we can nest `ParallelChain`s inside `ParallelChain`s

In [1]:
import pprint
import time

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ParallelChain

In [ ]:
"""
In Jupyter notebooks, the default event loop policy is set to asyncio.get_event_loop_policy().ipython_kernel, 
which is different from the default event loop policy used by the Python interpreter. 
This can cause issues if we're running code that assumes the default event loop policy is being used.

To resolve this issue, we set the event loop policy in the Jupyter notebook to match the 
one used by the Python interpreter.
"""
import nest_asyncio

# Set the event loop policy to the default used by the Python interpreter
nest_asyncio.apply()

## Simple Example

In [2]:
llm = OpenAI(temperature=0.9)

input_variables=['product']

prompt_1 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good mission statement for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

prompt_3 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good slogan for a company that makes {product}?",
)
chain_3 = LLMChain(llm=llm, prompt=prompt_3)

prompt_4 = PromptTemplate(
    input_variables=input_variables,
    template="What are some core values for a company that makes {product}?",
)
chain_4 = LLMChain(llm=llm, prompt=prompt_4)

When the `concurrent` flag is set to `True`, we can run the child chains concurrently. The `concurrent` flag is set to `True` by default.

In [3]:
parallel_chain = ParallelChain(
    input_variables=input_variables,
    chains={
        'name': chain_1, 
        'mission': chain_2, 
        'slogan': chain_3,
        'values': chain_4
    },
    verbose=True,
    concurrent=True
)
s = time.perf_counter()
output = parallel_chain("colorful socks")
pprint.pprint(output)
print('\033[1m' + f"Concurrent executed in {time.perf_counter()-s:0.2f} seconds." + '\033[0m')



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="mission" started.
Child chain for key="slogan" started.
Child chain for key="values" started.
Child chain for key="name" finished after 0.72 seconds.
Child chain for key="slogan" finished after 1.45 seconds.
Child chain for key="mission" finished after 1.94 seconds.
Child chain for key="values" finished after 5.79 seconds.

> Finished chain.
{'mission/text': '\n'
                 '\n'
                 'Our mission is to bring joy, vibrancy and self-expression to '
                 'everyday life by creating unique, colorful socks that make a '
                 'statement.',
 'name/text': '\n\nHappy Sock Co.',
 'product': 'colorful socks',
 'slogan/text': '\n\n"Life\'s Too Short to Wear Boring Socks!"',
 'values/text': '\n'
                '\n'
                '1. Quality: Producing high-quality, stylish, and durable '
                'socks.\n'
                '2. Creativity: Striving to cr

Setting the `concurrent` flag to `False` would run the child chains serially.

In [4]:
parallel_chain.concurrent=False
s = time.perf_counter()
output = parallel_chain("colorful socks")
pprint.pprint(output)
print('\033[1m' + f"Serial executed in {time.perf_counter()-s:0.2f} seconds." + '\033[0m')



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="name" finished after 0.59 seconds.
Child chain for key="mission" started.
Child chain for key="mission" finished after 2.59 seconds.
Child chain for key="slogan" started.
Child chain for key="slogan" finished after 0.52 seconds.
Child chain for key="values" started.
Child chain for key="values" finished after 5.32 seconds.

> Finished chain.
{'mission/text': '\n'
                 '\n'
                 'Our mission at [Company Name] is to provide affordable, '
                 'colorful, and quality socks that bring joy, comfort, and '
                 'warmth to our customers. We believe in creating a fun and '
                 'vibrant shopping experience for our customers, both online '
                 'and in-store.',
 'name/text': '\n\nSocktastic!',
 'product': 'colorful socks',
 'slogan/text': '\n\n"Put Some Color Into Your Step!"',
 'values/text': ' \n'
                '\n'
         

## Nesting `ParallelChain`s
It is possible to nest `ParallelChain`s inside one another. Continuing from the previous example, we nest a concurrent `ParallelChain` inside the previous serial `ParallelChain`.

In [5]:
prompt_5_1 = PromptTemplate(
    input_variables=input_variables,
    template="Which gift would go well with {product}?",
)
chain_5_1 = LLMChain(llm=llm, prompt=prompt_5_1)

prompt_5_2 = PromptTemplate(
    input_variables=input_variables,
    template="What gift would not go well with {product}?",
)
chain_5_2 = LLMChain(llm=llm, prompt=prompt_5_2)

chain_5 = ParallelChain(
    input_variables=input_variables,
    chains={'good_gift': chain_5_1, 'bad_gift': chain_5_2},
    verbose=True,
    concurrent=True
)

parallel_chain.chains.update({'gift': chain_5})

output = parallel_chain("colorful socks")
pprint.pprint(output)



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="name" finished after 0.62 seconds.
Child chain for key="mission" started.
Child chain for key="mission" finished after 1.83 seconds.
Child chain for key="slogan" started.
Child chain for key="slogan" finished after 1.20 seconds.
Child chain for key="values" started.
Child chain for key="values" finished after 2.17 seconds.
Child chain for key="gift" started.


> Entering new ParallelChain chain...
Child chain for key="good_gift" started.
Child chain for key="bad_gift" started.
Child chain for key="bad_gift" finished after 1.10 seconds.
Child chain for key="good_gift" finished after 1.51 seconds.

> Finished chain.
Child chain for key="gift" finished after 1.51 seconds.

> Finished chain.
{'gift/bad_gift/text': '\n'
                       '\n'
                       'Tools, such as a hammer or screwdriver, would not go '
                       'well with colorful socks.',
 'gift/good_gift/te

We can now make the outer `ParallelChain` execute concurrently again by setting `parallel_chain.concurrent=True`. Executing this nested chain will result in executing a concurrent `ParallelChain` inside another concurrent `ParallelChain`.

In [6]:
parallel_chain.concurrent=True
output = parallel_chain("colorful socks")
pprint.pprint(output)



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="mission" started.
Child chain for key="slogan" started.
Child chain for key="values" started.
Child chain for key="gift" started.


> Entering new ParallelChain chain...
Child chain for key="good_gift" started.
Child chain for key="bad_gift" started.
Child chain for key="good_gift" finished after 0.93 seconds.Child chain for key="name" finished after 0.94 seconds.

Child chain for key="bad_gift" finished after 0.93 seconds.

> Finished chain.
Child chain for key="gift" finished after 0.93 seconds.
Child chain for key="slogan" finished after 1.07 seconds.
Child chain for key="mission" finished after 2.37 seconds.
Child chain for key="values" finished after 5.71 seconds.

> Finished chain.
{'gift/bad_gift/text': '\n\nA vacuum cleaner.',
 'gift/good_gift/text': '\n'
                        '\n'
                        'A pair of colorful sneakers would pair nicely with '
                      